In [1]:
#hamiltonian definition and gate generation

In [2]:
#hamiltonian generation
#1D Isotropic Heisenberg open chain ; aperiodic
from cudaq import spin
nspins = 5
J = 1.0
ham = 0
for gt in range(nspins-1):
    print(gt)
    ham = ham + J*(spin.x(gt)*spin.x(gt+1) + spin.y(gt)*spin.y(gt+1) + spin.z(gt)*spin.z(gt+1))

0
1
2
3


In [28]:
for term in ham:
    print(term.get_pauli_word())
    print(term.get_raw_data())

XX
([[True, True, False, False]], [(1+0j)])
YY
([[True, True, True, True]], [(1+0j)])
ZZ
([[False, False, True, True]], [(1+0j)])

([[]], [0j])
XX
([[False, True, True, False, False, False]], [(1+0j)])
YY
([[False, True, True, False, True, True]], [(1+0j)])
ZZ
([[False, False, False, False, True, True]], [(1+0j)])
XX
([[False, False, True, True, False, False, False, False]], [(1+0j)])
YY
([[False, False, True, True, False, False, True, True]], [(1+0j)])
ZZ
([[False, False, False, False, False, False, True, True]], [(1+0j)])
XX
([[False, False, False, True, True, False, False, False, False, False]], [(1+0j)])
YY
([[False, False, False, True, True, False, False, False, True, True]], [(1+0j)])
ZZ
([[False, False, False, False, False, False, False, False, True, True]], [(1+0j)])


/tmp/ipykernel_22554/4258056359.py:3: DeprecationWarning: raw data access will no longer be supported
  print(term.get_raw_data())


In [4]:
nqbt = nspins

In [5]:
import cudaq

#state preparation circuit
@cudaq.kernel()
def state_prep():
    q = cudaq.qvector(nqbt)
    for i in range(nqbt):
        h(q[i])
    for i in range(0, nqbt-1, 2):
        x.ctrl(q[i], q[i+1])


In [6]:
print(cudaq.draw("ascii", state_prep))

     ╭───╮     
q0 : ┤ h ├──●──
     ├───┤╭─┴─╮
q1 : ┤ h ├┤ x ├
     ├───┤╰───╯
q2 : ┤ h ├──●──
     ├───┤╭─┴─╮
q3 : ┤ h ├┤ x ├
     ├───┤╰───╯
q4 : ┤ h ├─────
     ╰───╯     



In [ ]:
import numpy as np
#defining the XX(theta) gate
@cudaq.kernel()
def xx_gate(q0 : cudaq.qubit, q1 : cudaq.qubit, theta : float):
    h(q0)
    h(q1)
    x.ctrl(q0, q1)
    rz(-2*theta, q1)
    x.ctrl(q0, q1)
    h(q0)
    h(q1)

#defining the ZZ(theta) gate
@cudaq.kernel()
def zz_gate(q0 : cudaq.qubit, q1 : cudaq.qubit, theta : float):
    x.ctrl(q0, q1)
    rz(-2*theta, q1)
    x.ctrl(q0, q1)

#defining the YY(theta) gate
@cudaq.kernel()
def yy_gate(q0 : cudaq.qubit, q1 : cudaq.qubit, theta : float):
    rz(np.pi/2, q0)
    rz(np.pi/2, q1)
    h(q0)
    h(q1)
    x.ctrl(q0, q1)
    rz(-2*theta, q1)
    x.ctrl(q0, q1)
    h(q0)
    h(q1)
    rz(-np.pi/2, q0)
    rz(-np.pi/2, q1)
    

In [ ]:
# ansatz circuit
@cudaq.kernel()
def ansatz(thetas: list[float]):
    thetaas = iter(thetas)
    q = cudaq.qvector(nqbt)
    for term in ham:
    

In [18]:
print(cudaq.draw("ascii", ansatz, [0]))

     ╭───╮╭───╮                   ╭───╮╭───╮
q0 : ┤ x ├┤ h ├──●─────────────●──┤ h ├┤ x ├
     ├───┤├───┤╭─┴─╮╭───────╮╭─┴─╮├───┤├───┤
q1 : ┤ x ├┤ h ├┤ x ├┤ rz(0) ├┤ x ├┤ h ├┤ x ├
     ╰───╯╰───╯╰───╯╰───────╯╰───╯╰───╯╰───╯

